### **Cambio de User-Agent**

Seguimos trabajando en el enlace:

https://www.audible.com/search

Hasta ahora, hemos raspado todas las páginas de este sitio web, sin embargo, este sitio web ya sabe que estamos usando Scrapy porque estamos enviando requests usando el **`user agent`** por defecto. Ahora, voy a mostrarte cómo ver el user agent en Scrapy. 

Así que en primer lugar, abriremos el terminal y escribiremos este comando **`scrapy shell`**, luego dentro de comillas, a la derecha, el enlace de la página web **`https://www.audible.com/search`** y pulsaremos enter.

<center><img src="https://i.postimg.cc/C1Zxp2cH/ws-121.png"></center>

Ahora para obtener el user agent, tenemos que escribir **`request.headers`**. Luego presiono enter y vemos aquí un diccionario. Y una de las keys de este diccionario es **`user agent`**. Así que aquí está el user agent, en el value dicho a este user agent tiene la palabra **`Scrapy`**. Así que con esto, el sitio web sabrá fácilmente que estábamos usando Scrapy para scrapear el sitio web. Esto no es una buena práctica. Así que lo que tenemos que hacer es cambiar el user agent para que parezca que estamos enviando la petición usando chrome.

<center><img src="https://i.postimg.cc/26P8kxJp/ws-122.png"></center>

Primero voy a mostrarte **`cómo obtener el user agent que estás usando actualmente en tu navegador`**:

<center><img src="https://i.postimg.cc/prPpV1w9/ws-123.png"></center>

Primero seleccionamos **`Red`** (Network) y luego recargamos el sitio:

<center><img src="https://i.postimg.cc/x8gqX3Q6/ws-124.png"></center>

Filtramos por la palabra **`html`** y luego seleccionamos el elemento que aparece indicado:

<center><img src="https://i.postimg.cc/DfcSpnG1/ws-125.png"></center>

Seleccionamos **`Encabezados`** (Headers) y buscamos nuestro **`User-Agent`**:

<center><img src="https://i.postimg.cc/WbbhQCmC/ws-126.png"></center>

Existen 3 formas de fijar el User-Agent que estamos utilizando:

**PRIMERA FORMA**

Nos dirigimos al archivo **`settings.py`** y editamos USER-AGENT: 

<center><img src="https://i.postimg.cc/4yb5SqgB/ws-129.png"></center>
<center><img src="https://i.postimg.cc/Pfm2JpwY/ws-130.png"></center>

**SEGUNDA FORMA**

Nos dirigimos al archivo **`settings.py`** y editamos DEFAULT_REQUEST_HEADERS: 

<center><img src="https://i.postimg.cc/sXWKp10F/ws-127.png"></center>
<center><img src="https://i.postimg.cc/90jb1VvD/ws-128.png"></center>

**TERCERA FORMA**

Es indicarlo dentro del script y asi es como lo haremos.

Utilizamos el **`User-Agent de Chrome`**:

**`Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36`**

In [ ]:
import scrapy


class AudibleSpider(scrapy.Spider):
    name = 'audible'
    allowed_domains = ['www.audible.com']
    start_urls = ['https://www.audible.com/search/']

    def start_requests(self):
        # Editar los headers por defecto (user-agent)
        yield scrapy.Request(url='https://www.audible.com/search/', callback=self.parse,
                       headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'})

    def parse(self, response):
        # Obtener la caja que contiene toda la información que queremos (título, autor, longitud)
        product_container = response.xpath('//li[contains(@class , "productListItem")]')

        # Recorrer en bucle cada uno de los productos enumerados en la caja product_container
        for product in product_container:
            book_title = product.xpath('.//h3[contains(@class , "bc-heading")]/a/text()').get()
            book_author = product.xpath('.//li[contains(@class , "authorLabel")]/span/a/text()').getall()
            book_length = product.xpath('.//li[contains(@class , "runtimeLabel")]/span/text()').get()

            # Devuelve los datos extraídos y también el user agent definido anteriormente
            yield {
                'title':book_title,
                'author':book_author,
                'length':book_length,
                # Retornamos el user-agent solo para comprobar que esta correcto el que agregamos
                'User-Agent':response.request.headers['User-Agent'], 
            }

        # Obtener la barra de paginación (pagination) y luego el enlace dentro del botón de página siguiente (next_page_url)
        pagination = response.xpath('//ul[contains(@class , "pagingElements")]')
        next_page_url = pagination.xpath('.//span[contains(@class , "nextButton")]/a/@href').get()

        # Ir al enlace "next_page_url" usando el user agent definido antes
        if next_page_url:
            yield response.follow(url=next_page_url, callback=self.parse,
                                  headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'})


Por tanto, si ahora lo ejecutamos por consola:

<center><img src="https://i.postimg.cc/6pTxRJnr/ws-132.png"></center>
<center><img src="https://i.postimg.cc/3x65FSFP/ws-133.png"></center>